For scraping data from the Global Footprint Network.
https://www.footprintnetwork.org/

Countrycode Canada: 33

data types: 
biocapacity per capita (gha / person): 'BCpc'
biocapacity (gha): 'BCtot'
ecological footprint per capita (gha / person): 'EFCpc'
ecological footprint (gha): 'EFCtot'
number of earths: 'earth'


In [150]:
import pandas as pd
import requests
import json

In [151]:
def download(country, datastring): 
    
    countrycode = {'Canada':33,
                    'World':5001}[country]
    headers = {
        'Accept': 'application/json',
    }
    key = '13CMLmdbBei8dBCKf4tv3Gq7u2Vkg89so04K2reij4tm0l8o17r6'
    url_base = 'https://api.footprintnetwork.org/v1/data/'

    yearout = []
    valueout = []
    years = list(range(1961,2017))

    for year in years:
        url = url_base+str(countrycode)+'/'+str(year)+'/'+datastring
        r = requests.get(url, headers=headers, auth=('any-user-name', key))
        yearout.append((json.loads(r.content))[0]['year'])
        valueout.append((json.loads(r.content))[0]['value'])

    df = pd.DataFrame({'year':yearout, datastring+'_'+country:valueout})
    return df 


In [152]:
def mergedata(country):
  
    df = pd.DataFrame({'year':list(range(1961,2017))})
    
    for datastring in ['BCpc','BCtot','EFCpc','EFCtot','earth']:
        dg = download(country,datastring)
        df = df.merge(dg, on='year', how='left')
        
    return df

In [153]:
def mergecountries():
    
    df = pd.DataFrame({'year':list(range(1961,2017))})
    
    for country in ['Canada', 'World']:
        dg = mergedata(country)
        df = df.merge(dg, on = 'year', how='left')
        
    return df


In [156]:
df = mergecountries()

In [207]:
df.to_csv('~/code/footprint/static/footprintnetwork.csv')
#df=pd.read_csv('~/code/footprint/static/footprintnetwork.csv')

In [169]:
import datetime as dt

df['od_world'] = (df['BCtot_World'] / df['EFCtot_World']) * 365
df['od_can_pc'] = (df['BCpc_World']/ df['EFCpc_Canada']) * 365
df['odate_world'] = [pd.to_datetime('1/1/1900') + dt.timedelta(days=i) for i in df['od_world']]
df['odate_can'] = [pd.to_datetime('1/1/1900') + dt.timedelta(days=i) for i in df['od_can_pc']]

# to do: deal with leap years

In [180]:
type(df['odate_world'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [206]:
df.tail()

,year,BCpc_Canada,BCtot_Canada,EFCpc_Canada,EFCtot_Canada,earth_Canada,BCpc_World,BCtot_World,EFCpc_World,EFCtot_World,earth_World,od_world,od_can_pc,odate_world,odate_can
51,2012,15.452522,5.393038e+08,8.271601,2.886846e+08,4.963607,1.666450,1.187875e+10,2.822215,2.011725e+10,1.693550,215.523731,73.535236,1900-08-04 12:34:10.357373,1900-03-15 12:50:44.403812
52,2013,15.879022,5.598229e+08,8.748133,3.084198e+08,5.224800,1.674348,1.207779e+10,2.851806,2.057129e+10,1.703233,214.298389,69.859139,1900-08-03 07:09:40.814431,1900-03-11 20:37:09.645206
53,2014,15.212381,5.416322e+08,7.765584,2.764913e+08,4.661197,1.666006,1.215926e+10,2.824153,2.061194e+10,1.695163,215.318418,78.306066,1900-08-04 07:38:31.343064,1900-03-20 07:20:44.094690
54,2015,15.163473,5.451223e+08,7.757967,2.788966e+08,4.714809,1.645447,1.214835e+10,2.777221,2.050424e+10,1.687822,216.255088,77.415643,1900-08-05 06:07:19.635674,1900-03-19 09:58:31.552737
55,2016,15.122392,5.487886e+08,7.739935,2.808807e+08,4.749153,1.629751,1.216928e+10,2.746636,2.050903e+10,1.685310,216.577182,76.855816,1900-08-05 13:51:08.509097,1900-03-18 20:32:22.514225


In [205]:
# plotly implementation
import plotly
import plotly.graph_objects as go
import plotly.figure_factory as ff

fig = go.Figure(
        data = [
            go.Bar(
                x=df['year'],
                y=df['od_can_pc']
                  ),
            go.Bar(
                x=df['year'],
                y=df['od_world']
                )
              ]
    )
#fig.update_yaxes(autorange="reversed")


fig.show()